# 데이터 분석의 장벽을 낮추는 판다스 핀셋 강의

LG 에러 탐지 대회 코드 리뷰

# Library & Data Import

In [11]:
import warnings
warnings.filterwarnings(action='ignore')

import os, sys
import time
import datetime

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc, roc_auc_score, roc_curve, precision_recall_curve, recall_score, precision_score

import lightgbm
from lightgbm import LGBMClassifier
import catboost
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import StratifiedKFold

In [12]:
print(np.__version__)
print(pd.__version__)
print(sklearn.__version__)
print(lightgbm.__version__)
print(catboost.__version__)

1.19.3
1.0.5
0.23.2
3.0.0
0.24.1


In [13]:
TRAIN_P_PATH = r'C:\Users\Wyatt\wyatt37/Data/systemError/train_problem_data.csv'
TRAIN_Q_PATH = r'C:\Users\Wyatt\wyatt37/Data/systemError/train_quality_data.csv'
TRAIN_E_PATH = r'C:\Users\Wyatt\wyatt37/Data/systemError/train_err_data.csv'
TEST_Q_PATH = r'C:\Users\Wyatt\wyatt37/Data/systemError/test_quality_data.csv'
TEST_E_PATH = r'C:\Users\Wyatt\wyatt37/Data/systemError/test_err_data.csv'
SUBMISSION_PATH = r'C:\Users\Wyatt\wyatt37/Data/systemError/sample_submission.csv'

In [14]:
%%time
train_p = pd.read_csv(TRAIN_P_PATH)
train_q = pd.read_csv(TRAIN_Q_PATH)
train_e = pd.read_csv(TRAIN_E_PATH)
test_q = pd.read_csv(TEST_Q_PATH)
test_e = pd.read_csv(TEST_E_PATH)
submission = pd.read_csv(SUBMISSION_PATH)

Wall time: 14.6 s


In [15]:
train_p.shape, train_q.shape, train_e.shape, test_q.shape, test_e.shape, submission.shape

((5429, 2),
 (828624, 16),
 (16554663, 6),
 (747972, 16),
 (16532648, 6),
 (14999, 2))

In [16]:
train_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5429 entries, 0 to 5428
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   user_id  5429 non-null   int64
 1   time     5429 non-null   int64
dtypes: int64(2)
memory usage: 85.0 KB


In [17]:
train_q.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828624 entries, 0 to 828623
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   time        828624 non-null  int64  
 1   user_id     828624 non-null  int64  
 2   fwver       788544 non-null  object 
 3   quality_0   684192 non-null  float64
 4   quality_1   828624 non-null  int64  
 5   quality_2   788511 non-null  float64
 6   quality_3   828624 non-null  int64  
 7   quality_4   828624 non-null  int64  
 8   quality_5   828604 non-null  object 
 9   quality_6   828624 non-null  int64  
 10  quality_7   828624 non-null  object 
 11  quality_8   828624 non-null  object 
 12  quality_9   828624 non-null  object 
 13  quality_10  828624 non-null  object 
 14  quality_11  828624 non-null  int64  
 15  quality_12  828624 non-null  int64  
dtypes: float64(2), int64(8), object(6)
memory usage: 101.2+ MB


In [18]:
train_e.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16554663 entries, 0 to 16554662
Data columns (total 6 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   user_id   int64 
 1   time      int64 
 2   model_nm  object
 3   fwver     object
 4   errtype   int64 
 5   errcode   object
dtypes: int64(3), object(3)
memory usage: 757.8+ MB


In [19]:
test_q.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747972 entries, 0 to 747971
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   time        747972 non-null  int64  
 1   user_id     747972 non-null  int64  
 2   fwver       725208 non-null  object 
 3   quality_0   641388 non-null  float64
 4   quality_1   747961 non-null  object 
 5   quality_2   726857 non-null  float64
 6   quality_3   747972 non-null  int64  
 7   quality_4   747972 non-null  int64  
 8   quality_5   747928 non-null  object 
 9   quality_6   747972 non-null  int64  
 10  quality_7   747972 non-null  object 
 11  quality_8   747972 non-null  object 
 12  quality_9   747972 non-null  object 
 13  quality_10  747972 non-null  object 
 14  quality_11  747972 non-null  int64  
 15  quality_12  747972 non-null  int64  
dtypes: float64(2), int64(7), object(7)
memory usage: 91.3+ MB


In [20]:
test_e.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16532648 entries, 0 to 16532647
Data columns (total 6 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   user_id   int64 
 1   time      int64 
 2   model_nm  object
 3   fwver     object
 4   errtype   int64 
 5   errcode   object
dtypes: int64(3), object(3)
memory usage: 756.8+ MB


# Preprocessing

In [21]:
def preprocessing_problem(df, object_='binary'):
    """
    definition:
    train_problem 테이블을 받아서 target 값으로 변환
    1. {0, 1}의 binary로 변환
    2. {0 ~ n}의 multiclass로 변환
    """
    
    # 10001부터 24999까지의 index를 만들어줍니다.
    user_id_idx = np.array(range(10000, 25000, 1))
    
    # train_new_p라는 새로운 df를 만들고 index는 위에서 만든 user_id_idx 로 지정해줍니다.
    new_p = pd.DataFrame(index = user_id_idx)
    new_p['target'] = 0
    
    if object_ == 'binary':
        new_p.iloc[df.user_id.unique()-10000] = 1
        new_p = new_p.reset_index()
        new_p.rename({'index':'user_id'}, axis=1, inplace=True)
        
    elif object_ == 'multi':
        # multi는 count()로 집계를 해줍니다.
        new_p['target'] = df.groupby('user_id')['time'].count()
        new_p = new_p.fillna(0)
        new_p = new_p.reset_index()
        new_p.rename({'index':'user_id'}, axis=1, inplace=True)
        
    return new_p

In [22]:
train_b_p = preprocessing_problem(train_p, 'binary')
train_m_p = preprocessing_problem(train_p, 'multi')

In [23]:
def preprocessing_quality(df):
    """
    definition:
    EDA를 통해 알아낸 정보로 train_q, test_q를 정리해서 내뿜어줍니다.
    1. qaulity_3, quality_4 를 drop 합니다.(단일 value)
    2. qaulity_k 변수들을 정수로 encoding 합니다.
    """
    # 먼저 3, 4번을 drop 합니다.
    df.drop(['quality_3', 'quality_4'], axis=1, inplace=True)
    
    # qual 변수만 할당해주고, 정수로 형변환 해줍니다.
    columns = train_q.columns[train_q.columns.str.contains('quality')]
    # for문을 통해 각 column을 반복 작업해줍니다.
    for col in columns:
        if df[col].dtype == 'float64': # 기존에 float은 패스
            df[col] = df[col].fillna(-2)
        elif df[col].dtype == 'int64': # 기존에 int도 패스
            df[col] = df[col].fillna(-2)
            #print(col)
        else:
            # nan값이 있으면 float으로 갈 수 없으니 '-2' 으로 채워줍니다.
            df[col] = df[col].fillna('-2')
            df[col] = df[col].apply(lambda x: str(x).replace(',' , ''))
            df[col] = df[col].astype(np.float64)
        
    # fwver 에서 null 값이 꽤 있습니다. missing으로 채우겠습니다.
    df.fwver = df.fwver.fillna('missing')
        
    return df

In [24]:
train_q = preprocessing_quality(train_q)
test_q = preprocessing_quality(test_q)

In [25]:
def preprocessing_fwver(df):
    """
    definition:
    별건 아니고, e-set에 fwver 변수에서 '10' 이라는 값이 있는데, 이게 errtype이랑 겹쳐요.
    그래서 10을 -> 8.5.2 으로 바꿔줄 겁니다.
    굳이 이렇게 바꾸는 이유는, 해당 fw가 8.5.3버전과 같은 model_nm을 공유하기 때문입니다.
    """
    df.fwver = df.fwver.replace('10', '8.5.2')
        
    return df

In [26]:
train_e = preprocessing_fwver(train_e)
test_e = preprocessing_fwver(test_e)

In [27]:
def make_datetime(df):
    """
    definition:
    'time' column이 str로 되어 있으니, datetime으로 바꿔주는 함수입니다.
    다만 'time'양식이 pandas함수에 적용이 안되니, 강제로 슬라이싱해서 만들어줘야 합니다.
    
    """
    
    df['year'] = df['time'].apply(lambda x: str(x)[:4])
    df['month'] = df['time'].apply(lambda x: str(x)[4:6])
    df['day'] = df['time'].apply(lambda x: str(x)[6:8])
    df['hour'] = df['time'].apply(lambda x: str(x)[8:10])
    df['minute'] = '00' # minute을 넣어주지 않으면 datetime이 완성이 안되니, 00으로 넣어줍니다.
    
    df['time'] = pd.to_datetime(df.year + df.month + df.day + df.hour + df.minute)
    
    return df

In [28]:
%%time
train_p = make_datetime(train_p)
train_q = make_datetime(train_q)
test_q = make_datetime(test_q)
train_e = make_datetime(train_e)
test_e = make_datetime(test_e)

Wall time: 59.5 s


# EDA

## Null Data Check & Missing Value Check

In [41]:
def check_train_test_column_values(train, test, column):
    # 함수 정의: 두 데이터 테이블과 특정 컬럼을 집어 넣으면 value를 비교하여 출력하는 함수
    
    # Train/Test set의 입력 칼럼의 value를 set으로 받아줍니다.
    train_colset = set(train[column])
    test_colset  = set(test[column])

    # Train/Test-set 고유한 value의 개수를 구함.
    print(f"Train-set에 있는 고유한 value 개수 : {len(train_colset)}")
    print(f"Test-set에 있는 고유한 value 개수 : {len(test_colset)}")

    # Train/Test-set 모두에 포함되어있는 value를 구함.
    print('='* 80)
    common_colset = train_colset.intersection(test_colset)
    print(f"Train/Test-set에 공통으로 포함되어 있는 value 개수 : {len(common_colset)}")
    if len(common_colset) > 100: # value가 너무 많으면 어차피 보기 힘드므로 출력을 pass 합니다
        pass
    else:
        try: # int나 float은 sorted가 먹지 않기 때문에 try except로 넣어줍니다.
            print(f"Train/Test-set에 공통으로 포함되어 있는 value : {sorted(common_colset)}")
        except:
            print(f"Train/Test-set에 공통으로 포함되어 있는 value : {(common_colset)}")

    # Train-set에만 있는 value를 구함.
    print('='* 80)
    train_only_colset = train_colset.difference(test_colset)
    print(f"Train-set에만 있는 value는 총 {len(train_only_colset)} 개 입니다.")
    if len(train_only_colset) > 100:
        pass
    else:
        try:
            print(f"Train-set에만 있는 value는 : {sorted(train_only_colset)}")
        except:
            print(f"Train-set에만 있는 value는 : {(train_only_colset)}")

    # Test-set에만 있는 value를 구함.
    print('='* 80)
    test_only_colset = test_colset.difference(train_colset)
    print(f"Test-set에만 있는 value는 총 {len(test_only_colset)} 개 입니다.")
    if len(test_only_colset) > 100:
        pass
    else:
        try:
            print(f"Test-set에만 있는 value는 : {sorted(test_only_colset)}")
        except:
            print(f"Test-set에만 있는 value는 : {(test_only_colset)}")

In [43]:
check_train_test_column_values(train_e, test_e, 'fwver')

Train-set에 있는 고유한 value 개수 : 37
Test-set에 있는 고유한 value 개수 : 40
Train/Test-set에 공통으로 포함되어 있는 value 개수 : 31
Train/Test-set에 공통으로 포함되어 있는 value : ['03.11.1141', '03.11.1149', '03.11.1167', '04.16.3439', '04.16.3553', '04.16.3569', '04.16.3571', '04.22.1656', '04.22.1666', '04.22.1684', '04.22.1750', '04.22.1778', '04.33.1125', '04.33.1149', '04.33.1171', '04.33.1185', '04.33.1261', '04.73.2237', '04.73.2571', '04.82.1684', '04.82.1730', '04.82.1778', '05.15.2092', '05.15.2114', '05.15.2120', '05.15.2138', '05.15.3104', '05.66.3237', '05.66.3571', '8.5.2', '8.5.3']
Train-set에만 있는 value는 총 6 개 입니다.
Train-set에만 있는 value는 : ['04.16.2641', '04.16.3345', '04.22.1442', '04.33.1095', '05.15.2090', '05.15.2122']
Test-set에만 있는 value는 총 9 개 입니다.
Test-set에만 있는 value는 : ['04.22.1170', '04.22.1448', '04.22.1478', '04.22.1608', '04.22.1772', '04.73.2569', '04.73.2577', '10.22.1770', '10.22.1780']


## errcode 탐험하기

In [44]:
temp = train_e[['errtype', 'errcode']].drop_duplicates()

In [45]:
count = {}

for i in sorted(temp.errtype.unique()):

    print(i, "_unique: ", temp[temp.errtype == i]['errcode'].unique())
    count[i] = (len(temp[temp.errtype == i]['errcode'].unique()))

1 _unique:  ['0' 'P-44010' 'P-41011' 'P-41007 ' 'P-44010 ' 'P-41007' 'P-41011 '
 'P-41001']
2 _unique:  ['1' '0']
3 _unique:  ['1' '2' '0']
4 _unique:  ['0' '1']
5 _unique:  ['B-A8002' 'Q-64002' 'S-61001' 'U-81009' 'V-21008' '700001' 'S-64002'
 'J-30021' 'S-65002' 'Q-64001' 'H-51042' 'C-11017' 'H-51046' 'H-51049'
 'V-21002' 'V-21003' 'V-21004' 'V-21005' 'B-51042' 'M-99999' 'U-82024'
 'H-51048' '60045' 'U-82026' 'C-13053' 'C-14014' 'V-21010' 'J-20029'
 'J-30010' 'Y-00008' 'S-64000' 'En00409' 'E-59902' 'Q-73004' 'C-12032'
 'J-40011' 'U-82023' 'Q-73006' 'D-10011' 'S-65' 'M-51007' 'S-64001'
 'Y-00005' 'P_41007' '2638' 'CM a' nan '40013' 'D-99999' 'U-82004'
 'En00402' 'B-51049' 'C-11020' 'V-21007' 'M-51020' 'En00406' 'C-11087'
 'U-81000' 'Y-00004' 'C-13039' '0001' 'U-82020' 'http' 'P_41001' 'U-81014']
6 _unique:  ['1' '14']
7 _unique:  ['1' '14']
8 _unique:  ['PHONE_ERR' 'PUBLIC_ERR' '20']
9 _unique:  ['V-21002' 'V-21005' '1' 'C-14014' 'V-21008' 'C-12032' 'V-21004' 'C-13039'
 'C-11020']
10 

In [46]:
err_type_user_counts = []
for i in sorted(train_e.errtype.unique()):
    print(i, "_'s Unique User Counts: ", train_e[train_e.errtype == i].groupby('user_id').count().shape[0])
    err_type_user_counts.append(train_e[train_e.errtype == i].groupby('user_id').count().shape[0])

1 _'s Unique User Counts:  950
2 _'s Unique User Counts:  390
3 _'s Unique User Counts:  1679
4 _'s Unique User Counts:  6881
5 _'s Unique User Counts:  11010
6 _'s Unique User Counts:  6467
7 _'s Unique User Counts:  7648
8 _'s Unique User Counts:  64
9 _'s Unique User Counts:  63
10 _'s Unique User Counts:  2360
11 _'s Unique User Counts:  14767
12 _'s Unique User Counts:  14799
13 _'s Unique User Counts:  3711
14 _'s Unique User Counts:  7891
15 _'s Unique User Counts:  14545
16 _'s Unique User Counts:  14431
17 _'s Unique User Counts:  5747
18 _'s Unique User Counts:  1768
19 _'s Unique User Counts:  419
20 _'s Unique User Counts:  1546
21 _'s Unique User Counts:  381
22 _'s Unique User Counts:  10281
23 _'s Unique User Counts:  10046
24 _'s Unique User Counts:  2519
25 _'s Unique User Counts:  420
26 _'s Unique User Counts:  13026
27 _'s Unique User Counts:  2267
28 _'s Unique User Counts:  2275
30 _'s Unique User Counts:  375
31 _'s Unique User Counts:  11091
32 _'s Unique User C

## quality log 살펴보기

In [47]:
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    print(i,'번째 nunique: ' ,train_q['quality_{}'.format(i)].nunique())

0 번째 nunique:  754
1 번째 nunique:  31
2 번째 nunique:  799
5 번째 nunique:  4745
6 번째 nunique:  549
7 번째 nunique:  884
8 번째 nunique:  42
9 번째 nunique:  523
10 번째 nunique:  4200
11 번째 nunique:  12
12 번째 nunique:  14


In [48]:
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    print(i,'번째 nunique: ' ,test_q['quality_{}'.format(i)].nunique())

0 번째 nunique:  541
1 번째 nunique:  35
2 번째 nunique:  597
5 번째 nunique:  3995
6 번째 nunique:  528
7 번째 nunique:  839
8 번째 nunique:  44
9 번째 nunique:  431
10 번째 nunique:  3515
11 번째 nunique:  13
12 번째 nunique:  17


In [49]:
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    print(train_q['quality_{}'.format(i)].value_counts().head(10))

 0.0    542790
-2.0    144432
-1.0    130828
 1.0      2097
 2.0      1252
 3.0       518
 4.0       410
 5.0       385
 6.0       358
 7.0       314
Name: quality_0, dtype: int64
 0    670270
-1    153649
 1      2567
 2      1140
 3       391
 4       202
 5       134
 6        71
 7        50
 8        35
Name: quality_1, dtype: int64
 0.0    632469
-1.0    144392
-2.0     40113
 1.0      2937
 2.0      1073
 3.0       580
 4.0       455
 5.0       427
 6.0       386
 7.0       337
Name: quality_2, dtype: int64
 0.0    428096
-1.0    153354
 1.0     56171
 2.0     35978
 3.0     21596
 4.0     11204
 5.0      8536
 6.0      6836
 7.0      5506
 8.0      4495
Name: quality_5, dtype: int64
 0      662217
-1      153531
 600      1852
 1        1347
 5         644
 2         603
 3         475
 6         445
 4         408
 10        320
Name: quality_6, dtype: int64
0.0     757788
1.0       6768
2.0       3648
5.0       3456
6.0       2688
3.0       2568
4.0       2124
10.0      1632


# Feature Engineering

## from Error_log

### 사용한 model의 개수

In [50]:
train_model_count = train_e[['user_id', 'model_nm']].drop_duplicates().groupby('user_id').count()
test_model_count = test_e[['user_id', 'model_nm']].drop_duplicates().groupby('user_id').count()

### 사용한 fwver 의 개수

In [51]:
train_fwver_count = train_e[['user_id', 'fwver']].drop_duplicates().groupby('user_id').count()
test_fwver_count = test_e[['user_id', 'fwver']].drop_duplicates().groupby('user_id').count()

### 경험한 error 의 총 개수

In [52]:
train_err_count = train_e.groupby('user_id')['errcode'].count()
test_err_count = test_e.groupby('user_id')['errcode'].count()

### 경험한 각 errtype의 value별 개수

In [53]:
# 23호
train_errcode_23 = train_e[train_e.errtype == 23][['user_id', 'errcode', 'hour']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)
test_errcode_23 = test_e[test_e.errtype == 23][['user_id', 'errcode', 'hour']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)

train_errcode_23.columns = ['UNKNOWN', 'ACTIVE', 'connLMP', 'connESTA', 'connTO', 'connLOCAL', 'STANDBY', 'TERMINATE']
test_errcode_23.columns = ['UNKNOWN', 'ACTIVE', 'connLMP', 'connESTA', 'connTO', 'connLOCAL', 'STANDBY', 'TERMINATE']

In [54]:
# 31호
train_errcode_31 = train_e[train_e.errtype == 31][['user_id', 'errcode', 'time']].\
                            groupby(['user_id', 'errcode']).count().unstack().fillna(0)
test_errcode_31 = test_e[test_e.errtype == 31][['user_id', 'errcode', 'time']].\
                            groupby(['user_id', 'errcode']).count().unstack().fillna(0)
train_errcode_31.columns = ['err_31_0', 'err_31_1']
test_errcode_31.columns =['err_31_0', 'err_31_1']

In [55]:
# 33호
train_errcode_33 = train_e[train_e.errtype == 33][['user_id', 'errcode', 'hour']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)
test_errcode_33 = test_e[test_e.errtype == 33][['user_id', 'errcode', 'hour']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)

train_errcode_33.columns = ['err_33_1', 'err_33_2', 'err_33_3']
test_errcode_33.columns = ['err_33_1', 'err_33_2', 'err_33_3']

In [56]:
# 34호
train_errcode_34 = train_e[train_e.errtype == 34][['user_id', 'errcode', 'hour']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)
test_errcode_34 = test_e[test_e.errtype == 34][['user_id', 'errcode', 'hour']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)

train_errcode_34.columns = ['err_34_1', 'err_34_2', 'err_34_3', 'err_34_4', 'err_34_5', 'err_34_6']
test_errcode_34.columns = ['err_34_1', 'err_34_2', 'err_34_3', 'err_34_4', 'err_34_5', 'err_34_6']

### day max를 mean으로 나눈 것

In [57]:
%%time
train_e['day_2'] = train_e['time'].apply(lambda x: str(x)[:10])

train_meanDay = (train_e
                 .groupby(['user_id','day_2'])['day_2']
                 .count()
                 .unstack()
                 .fillna(0)
                 .loc[:, '2020-11-01':'2020-11-30']
                 .mean(axis=1))

train_maxDay = (train_e
                .groupby(['user_id','day_2'])['day_2']
                .count()
                .unstack()
                .fillna(0)
                .loc[:, '2020-11-01':'2020-11-30']
                .max(axis=1))

train_maxBymean = pd.Series(data = np.array(train_maxDay) / np.array(train_meanDay),
                            index = train_e.user_id.unique(),
                            name = 'mbm')

Wall time: 1min 17s


In [58]:
%%time
test_e['day_2'] = test_e['time'].apply(lambda x: str(x)[:10])

test_meanDay = (test_e
                .groupby(['user_id','day_2'])['day_2']
                .count()
                .unstack()
                .fillna(0)
                .loc[:, '2020-11-01':'2020-11-30']
                .mean(axis=1))

test_maxDay = (test_e
               .groupby(['user_id','day_2'])['day_2']
               .count()
               .unstack()
               .fillna(0)
               .loc[:, '2020-11-01':'2020-11-30']
               .max(axis=1))

test_maxBymean = pd.Series(data = np.array(test_maxDay) / np.array(test_meanDay),
                           index = test_e.user_id.unique(),
                           name = 'mbm')

Wall time: 1min 17s


## from Quality_log

### 각 quality의 표준편차

In [59]:
train_qual_std = train_q.groupby(['user_id']).std()
test_qual_std = test_q.groupby(['user_id']).std()

new_columns = ['q_std_0', 'q_std_1', 'q_std_2', 'q_std_5', 'q_std_6', 'q_std_7', 'q_std_8', 'q_std_9', 'q_std_10','q_std_11', 'q_std_12']

train_qual_std.columns = new_columns
test_qual_std.columns = new_columns

### 기록한 quality log의 개수(12개당 1번)

In [60]:
train_qual_log = train_q.groupby('user_id')['time'].count()/12
test_qual_log = test_q.groupby('user_id')['time'].count()/12

### quality당 순수 개수

In [61]:
# 값을 만들기 위해 새로운 df를 받아옵니다.
train_q_temp = train_q.copy()
test_q_temp = test_q.copy()

# 0 값을 전부 nan 값으로 바꿔줍니다.
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    train_q_temp.loc[train_q_temp['quality_{}'.format(i)] == 0, 'quality_{}'.format(i)] = np.nan
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    test_q_temp.loc[test_q_temp['quality_{}'.format(i)] == 0, 'quality_{}'.format(i)] = np.nan

# 필요없는 변수들을 버려줍니다.
train_q_temp.drop(['time', 'fwver', 'year', 'month', 'day', 'hour', 'minute'], axis=1, inplace=True)
test_q_temp.drop(['time', 'fwver', 'year', 'month', 'day', 'hour', 'minute'], axis=1, inplace=True)

# 그룹바이 카운트 해줍니다.
train_qual_counts = train_q_temp.groupby('user_id').count()
test_qual_counts = test_q_temp.groupby('user_id').count()

# 겹치는 컬럼명을 바꿔줍니다.
train_qual_counts.columns = ['q_c_0', 'q_c_1', 'q_c_2', 'q_c_5', 'q_c_6',
                             'q_c_7', 'q_c_8', 'q_c_9', 'q_c_10', 'q_c_11', 'q_c_12']
test_qual_counts.columns = ['q_c_0', 'q_c_1', 'q_c_2', 'q_c_5', 'q_c_6',
                             'q_c_7', 'q_c_8', 'q_c_9', 'q_c_10', 'q_c_11', 'q_c_12']

### quality당 음수, 0에 대한 count

In [62]:
train_q_temp = train_q.copy()
test_q_temp = test_q.copy()

In [63]:
train_q_temp.drop(['time', 'fwver', 'year', 'month', 'day', 'hour', 'minute'], axis=1, inplace=True)
test_q_temp.drop(['time', 'fwver', 'year', 'month', 'day', 'hour', 'minute'], axis=1, inplace=True)

In [64]:
# 0 값만 count를 위해서 음수와 양수를 전부 nan으로 만들겠습니다.

for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    train_q_temp.loc[train_q_temp['quality_{}'.format(i)] < 0, 'quality_{}'.format(i)] = np.nan
    
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    test_q_temp.loc[test_q_temp['quality_{}'.format(i)] < 0, 'quality_{}'.format(i)] = np.nan
    
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    train_q_temp.loc[train_q_temp['quality_{}'.format(i)] > 0, 'quality_{}'.format(i)] = np.nan
    
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    test_q_temp.loc[test_q_temp['quality_{}'.format(i)] > 0, 'quality_{}'.format(i)] = np.nan

In [65]:
train_qual_zeroCount = train_q_temp.groupby('user_id').count()
test_qual_zeroCount = test_q_temp.groupby('user_id').count()

In [66]:
new_columns = ['q_z_c_0', 'q_z_c_1', 'q_z_c_2', 'q_z_c_5', 'q_z_c_6', 'q_z_c_7', 'q_z_c_8', 'q_z_c_9', 'q_z_c_10','q_z_c_11', 'q_z_c_12']

train_qual_zeroCount.columns = new_columns
test_qual_zeroCount.columns = new_columns

In [67]:
train_q_temp = train_q.copy()
test_q_temp = test_q.copy()

In [68]:
train_q_temp.drop(['time', 'fwver', 'year', 'month', 'day', 'hour', 'minute'], axis=1, inplace=True)
test_q_temp.drop(['time', 'fwver', 'year', 'month', 'day', 'hour', 'minute'], axis=1, inplace=True)

In [69]:
# 음수 값만 count를 위해서 음수와 0을 전부 nan으로 만들겠습니다.

for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    train_q_temp.loc[train_q_temp['quality_{}'.format(i)] >= 0, 'quality_{}'.format(i)] = np.nan
    
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    test_q_temp.loc[test_q_temp['quality_{}'.format(i)] >= 0, 'quality_{}'.format(i)] = np.nan

In [70]:
train_qual_negaCount = train_q_temp.groupby('user_id').count()
test_qual_negaCount = test_q_temp.groupby('user_id').count()

In [71]:
new_columns = ['q_n_c_0', 'q_n_c_1', 'q_n_c_2', 'q_n_c_5', 'q_n_c_6', 'q_n_c_7', 'q_n_c_8', 'q_n_c_9', 'q_n_c_10','q_n_c_11', 'q_n_c_12']

train_qual_negaCount.columns = new_columns
test_qual_negaCount.columns = new_columns

### quality를 sum으로 groupby

In [72]:
train_each_quality_sum = train_q.groupby('user_id').sum().loc[:, 'quality_0':'quality_12']
test_each_quality_sum = test_q.groupby('user_id').sum().loc[:, 'quality_0':'quality_12']

quality_sum_colnms = ['quality_0_sum', 'quality_1_sum', 'quality_2_sum', 'quality_5_sum', 'quality_6_sum', 
                      'quality_7_sum', 'quality_8_sum', 'quality_9_sum','quality_10_sum', 'quality_11_sum', 
                      'quality_12_sum']

train_each_quality_sum.columns = quality_sum_colnms
test_each_quality_sum.columns = quality_sum_colnms

## from both

### time에 대한 유저별 표준편차

In [73]:
def time_to_seconds(x):
    return time.mktime(x.timetuple())

In [74]:
%%time
train_e['time_sec'] = train_e.time.apply(lambda x: time_to_seconds(x))
test_e['time_sec'] = test_e.time.apply(lambda x: time_to_seconds(x))
train_q['time_sec'] = train_q.time.apply(lambda x: time_to_seconds(x))
test_q['time_sec'] = test_q.time.apply(lambda x: time_to_seconds(x))

Wall time: 2min 9s


In [75]:
%%time
train_err_timestd = train_e.groupby(['user_id'])['time_sec'].std()
test_err_timestd = test_e.groupby(['user_id'])['time_sec'].std()
train_err_timestd = train_err_timestd.rename(level = 0, index = 't_e_std') 
test_err_timestd = test_err_timestd.rename(level = 0, index = 't_e_std') 

Wall time: 471 ms


In [76]:
%%time
train_qual_timestd = (train_q[['user_id', 'time_sec']].drop_duplicates()).groupby(['user_id']).std()
test_qual_timestd = (test_q[['user_id', 'time_sec']].drop_duplicates()).groupby(['user_id']).std()
train_qual_timestd.columns = ['t_q_std']
test_qual_timestd.columns = ['t_q_std']

Wall time: 123 ms


# Train & Predict

### Base set

In [77]:
# errtype을 유저별로 카운트 해줍니다.

X = train_e.groupby(['user_id', 'errtype'])['errcode'].count().unstack().fillna(0)
y = test_e.groupby(['user_id', 'errtype'])['errcode'].count().unstack().fillna(0)

X.columns = ['et_1', 'et_2', 'et_3', 'et_4', 'et_5', 'et_6', 'et_7', 'et_8', 'et_9', 'et_10', 'et_11', 'et_12', 'et_13',
 'et_14', 'et_15', 'et_16', 'et_17', 'et_18', 'et_19', 'et_20', 'et_21', 'et_22', 'et_23', 'et_24', 'et_25',
 'et_26', 'et_27', 'et_28', 'et_30', 'et_31', 'et_32', 'et_33', 'et_34', 'et_35', 'et_36', 'et_37', 'et_38',
 'et_39', 'et_40', 'et_41', 'et_42']
y.columns = ['et_1', 'et_2', 'et_3', 'et_4', 'et_5', 'et_6', 'et_7', 'et_8', 'et_9', 'et_10', 'et_11', 'et_12', 'et_13',
 'et_14', 'et_15', 'et_16', 'et_17', 'et_18', 'et_19', 'et_20', 'et_21', 'et_22', 'et_23', 'et_24', 'et_25',
 'et_26', 'et_27', 'et_28', 'et_30', 'et_31', 'et_32', 'et_33', 'et_34', 'et_35', 'et_36', 'et_37', 'et_38',
 'et_39', 'et_40', 'et_41', 'et_42']

In [78]:
# 하나 사라진 유저를 채워주는 코드입니다.
y = y.reindex(pd.RangeIndex(y.index.max() + 1)).ffill(0)[30000:]

In [79]:
X.shape, y.shape

((15000, 41), (14999, 41))

### Add Features

In [80]:
X = pd.concat([X,
               train_err_count, # 유저가 기록한 총 err수
               train_fwver_count, # 유저가 사용한 fw수
               train_model_count, # 유저가 사용한 model 수
               train_qual_std, # 각 퀄리티에 대한 유저별 편차
               train_qual_log, # 유저별 퀄리티 로그의 수
               train_errcode_23, # 23호 에러타입의 밸류별 개수
               train_errcode_33, # 33호 상동
               train_errcode_34, # 34호 상동
               train_qual_counts, # 각 퀄리티에서 0을 제외한 카운트
               train_qual_negaCount, # 각 퀄리티에 대해 음수만 카운트
               train_qual_zeroCount, # 각 퀄리티에 대해 0.만 카운트
               train_err_timestd,
               train_qual_timestd,
               train_each_quality_sum,
               train_errcode_31, # 31호 상동
               train_maxBymean # time
              ], axis=1).fillna(0)
X.shape

(15000, 122)

In [81]:
y = pd.concat([y,
               test_err_count, # 유저가 기록한 총 err수
               test_fwver_count, # 유저가 사용한 fw수
               test_model_count, # 유저가 사용한 model 수
               test_qual_std, # 각 퀄리티에 대한 유저별 편차
               test_qual_log, # 유저별 퀄리티 로그의 수
               test_errcode_23, # 23호 에러타입의 밸류별 개수
               test_errcode_33, # 33호 상동
               test_errcode_34, # 34호 상동
               test_qual_counts, # 각 퀄리티에서 0을 제외한 카운트
               test_qual_negaCount, # 각 퀄리티에 대해 음수만 카운트
               test_qual_zeroCount, # 각 퀄리티에 대해 0.만 카운트
               test_err_timestd,
               test_qual_timestd,
               test_each_quality_sum,
               test_errcode_31, # 31호 상동
               test_maxBymean, # time
              ], axis=1).fillna(0)
y.shape

(14999, 122)

In [82]:
# 일부 다중공선성을 띄는 변수들을 제거해줍니다.
X.drop(['et_20', 'et_36'], axis=1, inplace = True)
y.drop(['et_20', 'et_36'], axis=1, inplace = True)

X.shape, y.shape

((15000, 120), (14999, 120))

## Model

### Tuned Catb

In [83]:
def catb_fold_train_pred(train_x, train_y):

    # Train
    models     = []
    recalls    = []
    precisions = []
    auc_scores   = []
    threshold = 0.5
    # 파라미터 설정
    params =      {
                    'nan_mode': 'Min',
                    'eval_metric': 'Logloss',
                    'iterations': 1000,
                    'sampling_frequency': 'PerTree',
                    'leaf_estimation_method': 'Newton',
                    'grow_policy': 'SymmetricTree',
                    'penalties_coefficient': 1,
                    'boosting_type': 'Plain',
                    'model_shrink_mode': 'Constant',
                    'feature_border_type': 'GreedyLogSum',
                    'l2_leaf_reg': 3,
                    'random_strength': 1,
                    'rsm': 1,
                    'boost_from_average': False,
                    'model_size_reg': 0.5,
                    'subsample': 0.800000011920929,
                    'use_best_model': False,
                    'class_names': [0, 1],
                    'random_seed': 2584,
                    'depth': 6,
                    'posterior_sampling': False,
                    'border_count': 254,
                    'classes_count': 0,
                    'auto_class_weights': 'None',
                    'sparse_features_conflict_fraction': 0,
                    'leaf_estimation_backtracking': 'AnyImprovement',
                    'best_model_min_trees': 1,
                    'model_shrink_rate': 0,
                    'min_data_in_leaf': 1,
                    'loss_function': 'Logloss',
                    'learning_rate': 0.028116999194025993,
                    'score_function': 'Cosine',
                    'task_type': 'CPU',
                    'leaf_estimation_iterations': 10,
                    'bootstrap_type': 'MVS',
                    'max_leaves': 64
                    }
    #-------------------------------------------------------------------------------------
    # 5 Kfold cross validation
    s_fold = StratifiedKFold(n_splits=5, shuffle=True ,random_state=42)    

    for train_idx, val_idx in s_fold.split(train_x, train_y):

        # split train, validation set
        X = train_x.iloc[train_idx]
        y = train_y.iloc[train_idx]
        valid_x = train_x.iloc[val_idx]
        valid_y = train_y.iloc[val_idx]

        #run traning
        model = CatBoostClassifier(**params, verbose=0)
        model.fit(X, y)

        # cal valid prediction
        valid_prob = model.predict(valid_x)
        valid_pred = np.where(valid_prob > threshold, 1, 0)

        # cal scores
        recall    = recall_score(    valid_y, valid_pred)
        precision = precision_score( valid_y, valid_pred)
        auc_score = roc_auc_score(   valid_y, valid_prob)

        # append scores
        models.append(model)
        recalls.append(recall)
        precisions.append(precision)
        auc_scores.append(auc_score)
        
    return models, auc_scores, recalls, precisions

In [84]:
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = catb_fold_train_pred(X, train_b_p.target)
print(np.mean(auc_scores))

0.7301499999999999


In [85]:
pred_y_list = []
for model in models:
    pred_y = model.predict_proba(y)
    pred_y_list.append(pred_y[:, 1].reshape(-1,1))
    
pred_ensemble_catb = np.mean(pred_y_list, axis = 0)

In [86]:
pred_ensemble_catb

array([[0.93218016],
       [0.22254721],
       [0.48239791],
       ...,
       [0.70990508],
       [0.8953148 ],
       [0.43733988]])

### Tuned LGBM

In [87]:
def lgbm_fold_train_pred(train_x, train_y):

    # Train
    models     = []
    recalls    = []
    precisions = []
    auc_scores   = []
    threshold = 0.5
    # 파라미터 설정
    params =      {
                    'boosting_type':'gbdt', 
                    'class_weight':None,
                    'colsample_bytree':1.0,
                    'importance_type':'split',
                    'learning_rate':0.1,
                    'max_depth':-1,
                    'min_child_samples':20,
                    'min_child_weight':0.001,
                    'min_split_gain':0.0,
                    'n_estimators':100,
                    'n_jobs':-1,
                    'num_leaves':31,
                    'objective':None,
                    'random_state':2584,
                    'reg_alpha':0.0,
                    'reg_lambda':0.0,
                    'silent':True,
                    'subsample':1.0,
                    'subsample_for_bin':200000,
                    'subsample_freq':0
                    }
    #-------------------------------------------------------------------------------------
    # 5 Kfold cross validation
    s_fold = StratifiedKFold(n_splits=5, shuffle=True ,random_state=42)    

    for train_idx, val_idx in s_fold.split(train_x, train_y):

        # split train, validation set
        X = train_x.iloc[train_idx]
        y = train_y.iloc[train_idx]
        valid_x = train_x.iloc[val_idx]
        valid_y = train_y.iloc[val_idx]

        #run traning
        model = LGBMClassifier(**params)
        model.fit(X, y)

        # cal valid prediction
        valid_prob = model.predict(valid_x)
        valid_pred = np.where(valid_prob > threshold, 1, 0)

        # cal scores
        recall    = recall_score(    valid_y, valid_pred)
        precision = precision_score( valid_y, valid_pred)
        auc_score = roc_auc_score(   valid_y, valid_prob)

        # append scores
        models.append(model)
        recalls.append(recall)
        precisions.append(precision)
        auc_scores.append(auc_score)
        
    return models, auc_scores, recalls, precisions

In [88]:
models, auc_scores, _, _ = lgbm_fold_train_pred(X, train_b_p.target)
print(np.mean(auc_scores))

0.7339499999999999


In [89]:
pred_y_list = []
for model in models:
    pred_y = model.predict_proba(y)
    pred_y_list.append(pred_y[:, 1].reshape(-1,1))
    
pred_ensemble_lgbm = np.mean(pred_y_list, axis = 0)

In [90]:
pred_ensemble_lgbm

array([[0.94646884],
       [0.23926972],
       [0.47741895],
       ...,
       [0.78794031],
       [0.89074742],
       [0.42286365]])

### Tuned GBC

In [91]:
gbc = GradientBoostingClassifier(
                                ccp_alpha=0.0,
                                criterion='friedman_mse',
                                init=None,
                                learning_rate=0.1,
                                loss='deviance',
                                max_depth=3,
                                max_features=None,
                                max_leaf_nodes=None,
                                min_impurity_decrease=0.0,
                                min_impurity_split=None,
                                min_samples_leaf=1,
                                min_samples_split=2,
                                min_weight_fraction_leaf=0.0,
                                n_estimators=100,
                                n_iter_no_change=None,
                                presort='deprecated',
                                random_state=2584,
                                subsample=1.0,
                                tol=0.0001,
                                validation_fraction=0.1,
                                verbose=0,
                                warm_start=False
                            )

In [92]:
def gbc_fold_train_pred(train_x, train_y, N_SPLIT=5):

    # Train
    models     = []
    recalls    = []
    precisions = []
    auc_scores   = []
    threshold = 0.5

    k_fold = StratifiedKFold(n_splits=N_SPLIT, shuffle=True, random_state=42)
    for train_idx, val_idx in k_fold.split(train_x, train_y):

        # split train, validation set
        X = train_x.iloc[train_idx]
        y = train_y.iloc[train_idx]
        valid_x = train_x.iloc[val_idx]
        valid_y = train_y.iloc[val_idx]

        #run traning
        model = gbc.fit(X, y)

        # cal valid prediction
        valid_prob = model.predict(valid_x)
        valid_pred = np.where(valid_prob > threshold, 1, 0)

        # cal scores
        recall    = recall_score(    valid_y, valid_pred)
        precision = precision_score( valid_y, valid_pred)
        auc_score = roc_auc_score(   valid_y, valid_prob)
        
        # append scores
        models.append(model)
        recalls.append(recall)
        precisions.append(precision)
        auc_scores.append(auc_score)
    
    return models, auc_scores, recalls, precisions

In [93]:
%%time
models, auc_scores, _, _ = gbc_fold_train_pred(X, train_b_p.target)
print(np.mean(auc_scores))

0.72455
Wall time: 36.4 s


In [94]:
pred_y_list = []
for model in models:
    pred_y = model.predict_proba(y)
    pred_y_list.append(pred_y[:, 1].reshape(-1,1))
    
pred_ensemble_gbc = np.mean(pred_y_list, axis = 0)

In [95]:
pred_ensemble_gbc

array([[0.94729912],
       [0.20899576],
       [0.48470505],
       ...,
       [0.75559137],
       [0.87481054],
       [0.34822194]])

## Ensemble

In [96]:
pred_ensemble_catb

array([[0.93218016],
       [0.22254721],
       [0.48239791],
       ...,
       [0.70990508],
       [0.8953148 ],
       [0.43733988]])

In [97]:
pred_ensemble_lgbm

array([[0.94646884],
       [0.23926972],
       [0.47741895],
       ...,
       [0.78794031],
       [0.89074742],
       [0.42286365]])

In [98]:
pred_ensemble_gbc

array([[0.94729912],
       [0.20899576],
       [0.48470505],
       ...,
       [0.75559137],
       [0.87481054],
       [0.34822194]])

In [99]:
pred_ensemble_best = (
                            pred_ensemble_catb +
                            pred_ensemble_lgbm  +
                            pred_ensemble_gbc
                        ) / 3

submission.problem = pred_ensemble_best
submission

,user_id,problem
0,30000,0.941983
1,30001,0.223604
2,30002,0.481507
3,30003,0.790742
4,30004,0.897451
...,...,...
14994,44994,0.184165
14995,44995,0.289131
14996,44996,0.751146
14997,44997,0.886958


끝